# Binance PM Manual Console

In [1]:
from exli.enum_type import Exchange, AccountType, InstType, InstStatus
from exli.binanceunified.rest import BinanceUnifiedRestClient
from exli.binance.rest import BinanceSpotRestClient
from exli.binance.wrapper import BinanceRestWrapper
from exli.get_client import get_rest_client, get_rest_wrapper
from exli.inst_mngr import InstrumentManager, Instrument
from exli.utils import Cast, quantize

import pandas as pd
import json
import asyncio
from datetime import datetime
from decimal import Decimal
import random
import arrow
from pprint import pprint

In [2]:
console_acct = "bntest01"

scli: BinanceSpotRestClient = get_rest_client(
    Exchange.BINANCE,
    InstType.SP,
    account=console_acct,
)
pcli: BinanceUnifiedRestClient = get_rest_client(
    Exchange.BINANCEUNIFIED,
    InstType.LPS,
    account=console_acct,
)
pwpr_sp: BinanceRestWrapper = get_rest_wrapper(
    exchange=Exchange.BINANCEUNIFIED,
    acct_type=AccountType.MARGIN,
    account=console_acct,
)
pwpr_lps: BinanceRestWrapper = get_rest_wrapper(
    exchange=Exchange.BINANCEUNIFIED,
    acct_type=AccountType.LPS,
    account=console_acct,
)

inst_mngr = InstrumentManager()
await inst_mngr.init_instruments(Exchange.BINANCE, InstType.LPS)
await inst_mngr.init_instruments(Exchange.BINANCE, InstType.SP)
inst_d = inst_mngr.get_insts_by_exchange(Exchange.BINANCE, InstType.LPS)
print("Total LPS insts:", len(inst_d))

Total LPS insts: 489


# Account Overview

In [3]:
acct_sp_rsp = await scli.get_spot_account()
uid = acct_sp_rsp.get("uid", "null")
print("===={} (uid={}) PM get account raw resp====".format(console_acct, uid))
acct_rsp = await pcli.get_account()
pprint(acct_rsp)
print()

# 手续费
print("===={} account vip level @ {}====".format(console_acct, datetime.now().isoformat()))
vip_level_resp = await scli.raw_request("GET", "/sapi/v1/account/info", auth=True)
spot_fee_resp = await scli.sapi_get_commission_rate("SOLUSDT")
lps_fee_resp = await pcli.raw_request("GET", "/papi/v1/um/commissionRate", params={"symbol": "SOLUSDT"}, auth=True)
print(vip_level_resp)
print("Spot:", spot_fee_resp)
print("Swap:", lps_fee_resp)
print()

p_pos = await pcli.get_um_position_risk()
pos_df = pd.DataFrame(p_pos)
actual_eqty = float(acct_rsp["actualEquity"])
total_pos_value = pos_df.notional.astype(float).abs().sum() if "notional" in pos_df.columns else 0
util_rate = total_pos_value / actual_eqty if actual_eqty != 0 else 0
print("===={} PM account overview====".format(console_acct))
print("actual_eqty", actual_eqty)
print("uniMMR", acct_rsp["uniMMR"])
print("total_pos_value", total_pos_value)
print("util_rate: {:.2f}%".format(util_rate * 100))

====bntest01 (uid=1087907781) PM get account raw resp====
{'accountEquity': '0.0',
 'accountInitialMargin': '0.0',
 'accountMaintMargin': '0.0',
 'accountStatus': 'NORMAL',
 'actualEquity': '0.0',
 'totalAvailableBalance': '0.0',
 'totalMarginOpenLoss': '0.0',
 'uniMMR': '99999999',
 'updateTime': 1757645123700,
 'virtualMaxWithdrawAmount': '0.0'}

====bntest01 account vip level @ 2025-09-12T02:45:23.713631====
{'vipLevel': 0, 'isMarginEnabled': True, 'isFutureEnabled': True, 'isOptionsEnabled': True, 'isPortfolioMarginRetailEnabled': True}
Spot: [{'symbol': 'SOLUSDT', 'makerCommission': '0.001', 'takerCommission': '0.001'}]
Swap: {'symbol': 'SOLUSDT', 'makerCommissionRate': '0.000200', 'takerCommissionRate': '0.000500'}

====bntest01 PM account overview====
actual_eqty 0.0
uniMMR 99999999
total_pos_value 0
util_rate: 0.00%


In [4]:
# Set UM position side
resp = await pcli.raw_request("GET", "/papi/v1/um/positionSide/dual", auth=True)
print("UM position side", resp)

if resp["dualSidePosition"] == True:
    resp = await pcli.raw_request("POST", "/papi/v1/um/positionSide/dual", params={"dualSidePosition":"false"}, auth=True)
    print("Set UM dual position side to false, resp:", resp)

UM position side {'dualSidePosition': False}


In [5]:
# 查询自动偿还负余额模式
pm_auto_repay_neg_status = await pcli.raw_request("GET", "/papi/v1/repay-futures-switch", auth=True)
print("PM Auto Repay Neg Balance Switch:", pm_auto_repay_neg_status)

if pm_auto_repay_neg_status["autoRepay"] != True:
    # 开启自动偿还负余额模式
    pm_auto_repay_post_resp = await pcli.raw_request("POST", "/papi/v1/repay-futures-switch", params={"autoRepay": True}, auth=True)
    print("Turn on PM Auto Repay Neg Balance:", pm_auto_repay_post_resp)

PM Auto Repay Neg Balance Switch: {'autoRepay': True}


In [6]:
# repay cm/um negative balance
await pcli.raw_request("POST", "/papi/v1/repay-futures-negative-balance", params={}, auth=True)
# 统一账户资金归集
await pcli.raw_request("POST", "/papi/v1/auto-collection", auth=True)

{'msg': 'success'}

In [7]:
# view pm balance
p_bal_resp = await pcli.get_balance()
if p_bal_resp:
    p_bal_df = pd.DataFrame(p_bal_resp).set_index("asset")
    p_bal_df["updateTime"] = p_bal_df["updateTime"].apply(lambda x: arrow.get(x).format())
    p_bal_df["crossMarginFree"] = p_bal_df["crossMarginFree"].astype(float)
    p_bal_df["crossMarginAsset"] = p_bal_df["crossMarginAsset"].astype(float)
    p_bal_df["totalWalletBalance"] = p_bal_df["totalWalletBalance"].astype(float)
    p_bal_df = p_bal_df[p_bal_df.totalWalletBalance != 0]
else:
    p_bal_df = None
p_bal_df

,totalWalletBalance,crossMarginAsset,crossMarginBorrowed,crossMarginFree,crossMarginInterest,crossMarginLocked,umWalletBalance,umUnrealizedPNL,cmWalletBalance,cmUnrealizedPNL,updateTime,negativeBalance
asset,,,,,,,,,,,,


In [8]:
# USDT amt in PM account
ccy = "USDT"
if p_bal_df is not None and ccy in p_bal_df.index:
    print(json.dumps(p_bal_df.loc[ccy].to_dict(), indent=2))

In [9]:
# View PM UM Positions
p_pos = await pcli.get_um_position_risk()
if not p_pos:
    p_pos_df = None
    print("No positions!")
else:
    p_pos_df = pd.DataFrame(p_pos).set_index("symbol")
    p_pos_df["markPrice"] = p_pos_df["markPrice"].astype(float)
    p_pos_df["notional"] = p_pos_df["notional"].astype(float)
    p_pos_df["unRealizedProfit"] = p_pos_df["unRealizedProfit"].astype(float)
    p_pos_df["unpnl_rate"] = p_pos_df["unRealizedProfit"] / actual_eqty  # unpnl占total equity的比例
    p_pos_df["util_pct"] = p_pos_df["notional"].abs() / actual_eqty
    p_pos_df["updateTime"] = p_pos_df["updateTime"].apply(lambda x: arrow.get(x).format())
    p_pos_df = p_pos_df[
        [
            "positionAmt",
            "markPrice",
            "notional",
            "util_pct",
            # "maxNotionalValue",
            "unRealizedProfit",
            "unpnl_rate",
            "leverage",
            "positionSide",
            "updateTime",
        ]
    ].sort_values("notional")
p_pos_df

No positions!


In [10]:
symbol = "BTCUSDT"
if p_pos_df is not None and symbol in p_pos_df.index:
    print(json.dumps(p_pos_df.loc[symbol].to_dict(), indent=2))

In [11]:
# get spot account balances
resp = await scli.get_spot_account()
s_bal_df = pd.DataFrame(resp["balances"]).set_index("asset")
s_bal_df[s_bal_df["free"].astype(float) > 0]

,free,locked
asset,,
BNB,0.00026739,0.00000000
USDT,122.21869857,0.00000000
USD1,100.00000000,0.00000000


In [12]:
# CM Position
p_cm_pos = await pcli.get_cm_position_risk()
p_cm_pos_df = pd.DataFrame(p_cm_pos)
p_cm_pos_df

""


# Clear odds

In [13]:
max_odds_to_clear = 100

In [18]:
# Clear positions
pos_resp = await pcli.get_position_risk("linear")
df = pd.DataFrame(pos_resp)
df["positionAmt"] = df["positionAmt"].astype(float)
df["position_notl"] = df["positionAmt"] * df["markPrice"].astype(float)
df = df[df["position_notl"].abs().apply(lambda x: 0 < x < max_odds_to_clear)][
    ["symbol", "position_notl", "positionAmt"]
].sort_values("position_notl")
df["order_side"] = df["positionAmt"].apply(lambda amt: "SELL" if Decimal(amt) > 0 else "BUY")
df["order_type"] = "MARKET"
df["reduce_only"] = True
df["order_qty"] = df["positionAmt"].apply(lambda amt: str(abs(Decimal(str(amt)))))
df = df.reset_index(drop=True)
df

KeyError: 'positionAmt'

In [15]:
use_with_caution = True

for _idx, _row in df.iterrows():
    lps_orders_arg = {
        "symbol": _row["symbol"],
        "side": _row["order_side"],
        "type": _row["order_type"],
        "reduceOnly": _row["reduce_only"],
        "quantity": _row["order_qty"],
    }
    print(f"[{_idx+1}/{len(df)}] Order: {lps_orders_arg}")
    if use_with_caution:
        rsp = await pcli.place_um_order(**lps_orders_arg)
        if "code" not in rsp:
            print(f"[OK] {rsp}")
        else:
            print(f"[ERROR] {rsp}")
        await asyncio.sleep(0.5)

[1/1] Order: {'symbol': 'DOGEUSDT', 'side': 'SELL', 'type': 'MARKET', 'reduceOnly': True, 'quantity': '13.0'}
[OK] {'symbol': 'DOGEUSDT', 'side': 'SELL', 'executedQty': '0', 'orderId': 81469246535, 'goodTillDate': 0, 'avgPrice': '0.000000', 'origQty': '13', 'clientOrderId': 'avXuQmQ8AOal7dXabhz270', 'positionSide': 'BOTH', 'cumQty': '0', 'updateTime': 1757602959722, 'type': 'MARKET', 'reduceOnly': True, 'price': '0.000000', 'cumQuote': '0.000000', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'timeInForce': 'GTC', 'status': 'NEW', 'priceMatch': 'NONE'}


In [16]:
# Clear balances
def get_sp_order_size(size, min_order_size, lot_size, price, min_order_notional):
    size = float(Cast(size).quantize(lot_size, round_down=True))
    return size if (size > min_order_size and min_order_notional < size * price < max_odds_to_clear) else 0


bal_resp = await pcli.get_balance()
spot_balance_df = pd.DataFrame(bal_resp)
spot_balance_df["crossMarginFree"] = spot_balance_df["crossMarginFree"].astype(float)
spot_balance_df = spot_balance_df[spot_balance_df["crossMarginFree"] > 0][["asset", "crossMarginFree"]]
sp_inst_df = pd.DataFrame(
    [
        {
            "asset": inst.base_ccy,
            "symbol": inst.exchange_symbol,
            "lot_size": float(inst.lot_size),
            "min_order_size": float(inst.min_order_size),
            "min_order_notional": float(inst.min_order_notional),
        }
        for inst in inst_mngr.get_insts_by_exchange(Exchange.BINANCE, InstType.SP).values() if inst.quote_ccy=="USDT"
    ]
)
spot_df = pd.merge(spot_balance_df, sp_inst_df, how="left", on="asset")
px_resp = await scli.get_price()
px_ticker_df = pd.DataFrame(px_resp)
px_ticker_df["price"] = px_ticker_df["price"].astype(float)
spot_val_df = pd.merge(spot_df, px_ticker_df, how="left", on="symbol").dropna(subset=["price"])
spot_val_df["sp_notional"] = spot_val_df["crossMarginFree"] * spot_val_df["price"]
spot_val_df["order_quantity"] = spot_val_df.apply(
    lambda row: get_sp_order_size(
        row["crossMarginFree"],
        row["min_order_size"],
        row["lot_size"],
        row["price"],
        row["min_order_notional"],
    ),
    axis=1,
)
spot_val_df["order_notional"] = spot_val_df["order_quantity"] * spot_val_df["price"]
spot_val_df["order_side"] = "SELL"
spot_val_df["order_type"] = "MARKET"
spot_val_df = spot_val_df[(spot_val_df["order_quantity"] > 0) & (spot_val_df["asset"] != "BNB")]
spot_val_df = spot_val_df.reset_index(drop=True)
spot_val_df

,asset,crossMarginFree,symbol,lot_size,min_order_size,min_order_notional,price,sp_notional,order_quantity,order_notional,order_side,order_type


In [17]:
use_with_caution = True

for _idx, _row in spot_val_df.iterrows():
    sp_orders_arg = {
        "symbol": _row["symbol"],
        "side": _row["order_side"],
        "type": _row["order_type"],
        "quantity": _row["order_quantity"],
    }
    print(f"[{_idx+1}/{len(spot_val_df)}] Order: {sp_orders_arg}")
    if use_with_caution:
        order_resp = await pcli.place_margin_order(**sp_orders_arg)
        if "code" in order_resp:
            print("[ERROR] Order={}. Resp={}".format(sp_orders_arg, order_resp))
        else:
            print("[OK] Clear {}. OrderId={}".format(sp_orders_arg["symbol"], order_resp["clientOrderId"]))
        await asyncio.sleep(0.5)

# Check delta & reduce delta

In [ ]:
def to_sp_price_multiplier(exchange_symbol):
    inst = inst_mngr.get_inst_by_exchange_symbol(Exchange.BINANCE, InstType.SP, exchange_symbol=exchange_symbol)
    return None if not inst else inst.price_multiplier


def to_lps_price_multiplier(exchange_symbol):
    inst = inst_mngr.get_inst_by_exchange_symbol(Exchange.BINANCE, InstType.LPS, exchange_symbol=exchange_symbol)
    return None if not inst else inst.price_multiplier


sp_ticker_resp = await scli.get_price()
sp_ticker_df = pd.DataFrame(sp_ticker_resp)
sp_ticker_df["price"] = sp_ticker_df["price"].astype(float)
sp_ticker_df = sp_ticker_df.rename(columns={"symbol": "sp_ticker_symbol", "price": "sp_px"}, errors="raise")
sp_ticker_df["meta_symbol"] = sp_ticker_df["sp_ticker_symbol"].apply(
    lambda x: inst_mngr.get_meta_symbol_by_exchange_symbol(Exchange.BINANCE, InstType.SP, exchange_symbol=x),
)
sp_ticker_df["sp_price_multiplier"] = sp_ticker_df["sp_ticker_symbol"].apply(to_sp_price_multiplier)
sp_ticker_df = sp_ticker_df.dropna()

p_bal_resp = await pcli.get_balance()
p_bal_df = pd.DataFrame(p_bal_resp)
p_bal_df["total_sp_qty"] = p_bal_df["totalWalletBalance"].astype(float)
p_bal_df["crossMarginFree"] = p_bal_df["crossMarginFree"].astype(float)
p_bal_df["sp_ticker_symbol"] = p_bal_df["asset"].apply(lambda x: x + "USDT")
p_bal_df = p_bal_df[["asset", "total_sp_qty", "crossMarginFree", "sp_ticker_symbol"]]
p_bal_meta_df = pd.merge(left=p_bal_df, right=sp_ticker_df, how="left", on=["sp_ticker_symbol"])
p_bal_meta_df["sp_meta_qty"] = p_bal_meta_df["sp_price_multiplier"] * p_bal_meta_df["total_sp_qty"]

lps_pos = await pcli.get_um_position_risk()
lps_pos_df = pd.DataFrame(lps_pos)
lps_pos_df["pos_amt"] = lps_pos_df["positionAmt"].astype(float)
lps_pos_df["pos_notl"] = lps_pos_df["notional"].astype(float)
lps_pos_df["meta_symbol"] = lps_pos_df["symbol"].apply(
    lambda x: inst_mngr.get_meta_symbol_by_exchange_symbol(Exchange.BINANCE, InstType.LPS, exchange_symbol=x)
)
lps_pos_df = lps_pos_df[["symbol", "meta_symbol", "pos_amt", "pos_notl"]]

delta_df = pd.merge(p_bal_meta_df, lps_pos_df, on="meta_symbol", how="outer")
delta_df["lps_ticker_symbol"] = delta_df["meta_symbol"].apply(
    lambda x: inst_mngr.get_exchange_symbol_by_meta_symbol(Exchange.BINANCE, InstType.LPS, meta_symbol=x)
)
delta_df["lps_price_multiplier"] = delta_df["lps_ticker_symbol"].apply(to_lps_price_multiplier)
delta_df["pos_amt"] = delta_df["pos_amt"].fillna(0)
delta_df["lps_meta_qty"] = delta_df["lps_price_multiplier"] * delta_df["pos_amt"]
delta_df["total_sp_qty"] = delta_df["total_sp_qty"].fillna(0)
delta_df["sp_meta_qty"] = delta_df["sp_meta_qty"].fillna(0)
delta_df["delta_meta_qty"] = delta_df["sp_meta_qty"] + delta_df["lps_meta_qty"]
delta_df["delta_notl"] = delta_df["delta_meta_qty"] / delta_df["sp_price_multiplier"] * delta_df["sp_px"]
delta_df = delta_df[
    (delta_df["delta_meta_qty"] != 0)
    & (delta_df["delta_meta_qty"].notna())
    & (delta_df["delta_notl"].abs() > 10)
    & (
        delta_df["meta_symbol"].apply(
            lambda x: x
            not in [
                "BNB_USDT",
                # "BTC_USDT",
            ]
        )
    )
][
    [
        "meta_symbol",
        "asset",
        "sp_ticker_symbol",
        "total_sp_qty",
        "crossMarginFree",
        "sp_px",
        "sp_price_multiplier",
        "sp_meta_qty",
        "lps_ticker_symbol",
        "pos_amt",
        "pos_notl",
        "lps_price_multiplier",
        "lps_meta_qty",
        "delta_meta_qty",
        "delta_notl",
    ]
].set_index(
    "meta_symbol"
)

delta_df

In [113]:
_use_with_caution = True

for h_meta_symbol in delta_df.index.to_list():
    h_sp_inst = inst_mngr.get_inst_by_meta_symbol(Exchange.BINANCE, InstType.SP, meta_symbol=h_meta_symbol)
    h_lps_inst = inst_mngr.get_inst_by_meta_symbol(Exchange.BINANCE, InstType.LPS, meta_symbol=h_meta_symbol)
    print(f"************ meta_symbol={h_meta_symbol} ************")

    sp_qty = delta_df.loc[h_meta_symbol]["total_sp_qty"]
    lps_qty = delta_df.loc[h_meta_symbol]["pos_amt"]
    delta_meta_qty = delta_df.loc[h_meta_symbol]["delta_meta_qty"]
    delta_value = delta_df.loc[h_meta_symbol]["delta_notl"]
    print(f"sp_qty={sp_qty} lps_qty={lps_qty} delta_meta_qty={delta_meta_qty} delta_value={delta_value:.2f}")

    delta_qty_lps = Cast(float(delta_meta_qty / h_lps_inst.price_multiplier)).quantize(
        h_lps_inst.lot_size, round_down=True
    )
    order_qty_lps = quantize(abs(delta_qty_lps), h_lps_inst.lot_size, True)
    delta_qty_sp = Cast(float(delta_meta_qty / h_sp_inst.price_multiplier)).quantize(
        h_sp_inst.lot_size, round_down=True
    )
    order_qty_sp = quantize(abs(delta_qty_sp), h_sp_inst.lot_size, True)
    print(f"delta_qty_lps={delta_qty_lps} delta_qty_sp={delta_qty_sp}")

    if _use_with_caution:
        if delta_meta_qty < 0 and order_qty_lps > 0:
            rsp = await pcli.place_um_order(
                symbol=h_lps_inst.exchange_symbol,
                side="BUY",
                type="MARKET",
                quantity=str(order_qty_lps),
                reduceOnly=True,
            )
            print(rsp)
        elif delta_meta_qty > 0 and order_qty_sp > 0:
            rsp = await pcli.place_margin_order(
                symbol=h_sp_inst.exchange_symbol,
                side="SELL",
                type="MARKET",
                quantity=str(order_qty_sp),
            )
            print(rsp)

    print()
    await asyncio.sleep(0.5)

************ meta_symbol=BTC_USDT ************
sp_qty=0.01035328 lps_qty=0.0 delta_meta_qty=0.01035328 delta_value=1134.47
delta_qty_lps=0.010 delta_qty_sp=0.01035000
{'clientOrderId': 'NQkBUizy0snCnOteKwI72', 'cummulativeQuoteQty': '1134.10901250', 'executedQty': '0.01035000', 'fills': [{'commission': '0.00066162', 'commissionAsset': 'BNB', 'price': '109575.75000000', 'qty': '0.01035000'}], 'orderId': 44511738016, 'origQty': '0.01035000', 'price': '0.00000000', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'side': 'SELL', 'status': 'FILLED', 'symbol': 'BTCUSDT', 'timeInForce': 'GTC', 'transactTime': 1749607035527, 'type': 'MARKET'}



# BNB Dust

In [16]:
bal_resp = await pcli.get_balance()
spot_balance_df = pd.DataFrame(bal_resp)
spot_balance_df["crossMarginFree"] = spot_balance_df["crossMarginFree"].astype(float)
spot_balance_df = spot_balance_df[spot_balance_df["crossMarginFree"] > 0][["asset", "crossMarginFree"]]
sp_inst_df = pd.DataFrame(
    [
        {
            "asset": inst.base_ccy,
            "symbol": inst.exchange_symbol,
            "lot_size": float(inst.lot_size),
            "min_order_size": float(inst.min_order_size),
            "min_order_notional": float(inst.min_order_notional),
        }
        for inst in inst_mngr.get_insts_by_exchange(Exchange.BINANCE, InstType.SP).values()
        if inst.quote_ccy == "USDT"
    ]
)
spot_df = pd.merge(spot_balance_df, sp_inst_df, how="left", on="asset")
px_resp = await scli.get_price()
px_ticker_df = pd.DataFrame(px_resp)
px_ticker_df["price"] = px_ticker_df["price"].astype(float)
spot_val_df = pd.merge(spot_df, px_ticker_df, how="left", on="symbol").dropna(subset=["price"])
spot_val_df["notional"] = spot_val_df["crossMarginFree"] * spot_val_df["price"]
spot_val_df["will_dust"] = spot_val_df.apply(
    lambda row: (
        row["crossMarginFree"] < row["min_order_size"]
        or row["notional"] < 10
        or row["crossMarginFree"] * row["price"] < row["min_order_notional"]
    )
    and row["asset"] != "BNB",
    axis=1,
)
will_dust_df = (
    spot_val_df[spot_val_df["will_dust"]][["asset", "crossMarginFree", "symbol", "notional", "will_dust"]]
    .set_index("asset")
    .sort_values("notional", ascending=False)
)
will_dust_df

,crossMarginFree,symbol,notional,will_dust
asset,,,,
DOGE,0.586,DOGEUSDT,0.134522,True


In [17]:
# transfer dust assets from PM Margin -> Spot
use_with_caution = True

bal_resp = await pcli.get_balance()
trsf_bal_df = pd.DataFrame(bal_resp).set_index("asset")
for asset_key in will_dust_df.index.values:
    if float(trsf_bal_df.loc[asset_key]["crossMarginFree"]) > 0:
        print("Transfer {} {} from PM to SPOT".format(trsf_bal_df.loc[asset_key]["crossMarginFree"], asset_key))
        if use_with_caution:
            rsp = await scli.sapi_asset_transfer(
                type="PORTFOLIO_MARGIN_MAIN", asset=asset_key, amount=trsf_bal_df.loc[asset_key]["crossMarginFree"]
            )
            if "tranId" not in rsp:
                print("[ERROR]", rsp)
            await asyncio.sleep(0.5)

Transfer 0.586 DOGE from PM to SPOT


In [18]:
# dust preview
dust_preview_resp = await scli.sapi_asset_dustable()
dust_asset_l = list(map(lambda x: x["asset"], dust_preview_resp["details"]))
print(dust_asset_l)

['DOGE']


In [19]:
dust_resp = await scli.sapi_asset_dust(asset=",".join(dust_asset_l))
try:
    dustChargeRate = float(dust_resp["totalServiceCharge"]) / float(dust_resp["totalTransfered"])
    print("totalTransfered={}".format(dust_resp["totalTransfered"]))
    print("totalServiceCharge={}".format(dust_resp["totalServiceCharge"]))
    print("dustChargeRate={:.2f}%".format(dustChargeRate * 100))
except:
    pprint(dust_resp)

totalTransfered=0.00015633
totalServiceCharge=0.00000312
dustChargeRate=2.00%


In [20]:
# transfer all spot account BNB balance to PM
resp = await scli.get_spot_account()
s_bal_df = pd.DataFrame(resp["balances"]).set_index("asset")
sp_bnb_free = float(s_bal_df.loc["BNB"].free)
if sp_bnb_free > 0:
    print("Transfer {} BNB from spot to PM".format(sp_bnb_free))
    rsp = await scli.sapi_asset_transfer(type="MAIN_PORTFOLIO_MARGIN", asset="BNB", amount=str(sp_bnb_free))
    print(rsp)
else:
    print("No BNB in spot account.")

Transfer 0.00015321 BNB from spot to PM
{'tranId': 289204658849}


# Query orders

In [ ]:
# get sp order status
od = await pcli.papi_v1_margin_myTrades(symbol="FDUSDUSDT", limit=1000)
pd.DataFrame(data=od)

In [36]:
# get lps order status
od = await pcli.papi_v1_um_userTrades(symbol="ACTUSDT", orderId=2514755808)
pd.DataFrame(data=od)
# print(od)

,symbol,id,orderId,side,price,qty,realizedPnl,quoteQty,commission,commissionAsset,time,positionSide,maker,buyer
0,ACTUSDT,216328385,2514755808,BUY,0.1094110,1141044,133785.70147922,124842.7650840,0,USDT,1743503535616,BOTH,True,True


In [50]:
start_time = arrow.get("2025-04-01 10:32:15").int_timestamp * 1000
end_time = arrow.get("2025-04-01 12:32:15").int_timestamp * 1000

symbol_list = ["ACTUSDT"]
sp_order_rsp = await pwpr_sp.get_order_history(symbol_list=symbol_list, start_time=start_time, end_time=end_time)
df = pd.DataFrame(
    [
        {
            "create_time": dc.create_time,
            "symbol": dc.symbol,
            "side": dc.side.name,
            "order_id": dc.order_id,
            "client_id": dc.client_id,
            "quantity": float(dc.quantity),
            "filled_price": float(dc.filled_price),
            "filled_quantity": float(dc.filled_quantity),
            "order_type": dc.order_type.name,
            "time_in_force": dc.time_in_force.name,
            "status": dc.status.name,
        }
        for dc in sp_order_rsp["data"]["ACTUSDT"]
    ]
)

In [53]:
total_filled_value = (df["filled_price"] * df["filled_quantity"]).sum()
total_filled_quantity = df["filled_quantity"].sum()
avg_filled_price = total_filled_value / total_filled_quantity
print("total_filled_quantity={} avg_filled_price={}".format(total_filled_quantity, avg_filled_price))

total_filled_quantity=1162322.2 avg_filled_price=0.10652066992267722


# Batch set leverage

In [16]:
# instrument
im = InstrumentManager()
await im.init_instruments(Exchange.BINANCE, InstType.LPS)
inst_l = im.get_insts_by_exchange(Exchange.BINANCE, InstType.LPS)

for symbol, inst in inst_l.items():
    if inst.status == InstStatus.TRADING:
        rsp = await pcli.raw_request(
            "POST",
            "/papi/v1/um/leverage",
            {"symbol": inst.exchange_symbol, "leverage": 4},
            auth=True,
        )
        if "code" in rsp:
            print("[ERROR]", symbol, rsp)
        else:
            print("[OK]", symbol, rsp)
        await asyncio.sleep(0.1)

[OK] BTCUSDT {'leverage': 4, 'maxNotionalValue': '600000000', 'symbol': 'BTCUSDT'}
[OK] ETHUSDT {'leverage': 4, 'maxNotionalValue': '400000000', 'symbol': 'ETHUSDT'}
[OK] BCHUSDT {'leverage': 4, 'maxNotionalValue': '20000000', 'symbol': 'BCHUSDT'}
[OK] XRPUSDT {'leverage': 4, 'maxNotionalValue': '25000000', 'symbol': 'XRPUSDT'}
[OK] LTCUSDT {'leverage': 4, 'maxNotionalValue': '20000000', 'symbol': 'LTCUSDT'}
[OK] TRXUSDT {'leverage': 4, 'maxNotionalValue': '12000000', 'symbol': 'TRXUSDT'}
[OK] ETCUSDT {'leverage': 4, 'maxNotionalValue': '7000000', 'symbol': 'ETCUSDT'}
[OK] LINKUSDT {'leverage': 4, 'maxNotionalValue': '20000000', 'symbol': 'LINKUSDT'}
[OK] XLMUSDT {'leverage': 4, 'maxNotionalValue': '12500000', 'symbol': 'XLMUSDT'}
[OK] ADAUSDT {'leverage': 4, 'maxNotionalValue': '25000000', 'symbol': 'ADAUSDT'}
[OK] XMRUSDT {'leverage': 4, 'maxNotionalValue': '500000', 'symbol': 'XMRUSDT'}
[OK] DASHUSDT {'leverage': 4, 'maxNotionalValue': '500000', 'symbol': 'DASHUSDT'}
[OK] ZECUSDT {'

## Transfer

In [18]:
p_bal_resp = await pcli.get_balance()
p_bal_df = pd.DataFrame(p_bal_resp).set_index("asset")
p_bal_df.loc["USDT"]

totalWalletBalance      158.26241723
crossMarginAsset        158.26241723
crossMarginBorrowed              0.0
crossMarginFree         158.26241723
crossMarginInterest              0.0
crossMarginLocked                0.0
umWalletBalance                  0.0
umUnrealizedPNL          -0.78290781
cmWalletBalance                  0.0
cmUnrealizedPNL                  0.0
updateTime             1756039209241
negativeBalance                  0.0
Name: USDT, dtype: object

In [19]:
resp = await scli.get_spot_account()
s_bal_df = pd.DataFrame(resp["balances"]).set_index("asset")
s_bal_df.free = s_bal_df.free.astype(float)
s_bal_df[s_bal_df.free > 0]

,free,locked
asset,,
BNB,0.000351,0.00000000
DOGE,199.000000,0.00000000


In [85]:
# transfer some asset (ALL BALANCE!) from PM Margin -> Spot
trsf_asset = "PEPE"

_use_with_caution = False
if _use_with_caution:
    rsp = await scli.sapi_asset_transfer(
        type="PORTFOLIO_MARGIN_MAIN",
        asset=trsf_asset,
        amount=p_bal_df.loc[trsf_asset].totalWalletBalance,
    )
    print(rsp)

In [75]:
# transfer some asset from PM Margin -> Spot
rsp = await scli.sapi_asset_transfer(type="PORTFOLIO_MARGIN_MAIN", asset="USDT", amount=16122.58)
print(rsp)

{'tranId': 232660733113}


In [10]:
# transfer some asset from Spot -> PM Margin
rsp = await scli.sapi_asset_transfer(type="MAIN_PORTFOLIO_MARGIN", asset="DOGE", amount=100)
print(rsp)

{'tranId': 214154627817}


In [33]:
# transfer all asset from Spot -> PM Margin
resp = await scli.get_spot_account()
bal_df = pd.DataFrame(resp["balances"])

for _, _row in bal_df.iterrows():
    if float(_row["free"]) > 0:
        print("Transfer {} {} from spot to PM".format(_row["asset"], _row["free"]))
        rsp = await scli.sapi_asset_transfer(type="MAIN_PORTFOLIO_MARGIN", asset=_row["asset"], amount=_row["free"])
        print(rsp)

In [21]:
# Transfer all BNB from UM to MARGIN
p_bal_resp = await pcli.get_balance()
p_bal_df = pd.DataFrame(p_bal_resp).set_index("asset")
if float(p_bal_df.loc["BNB"]["umWalletBalance"]) > 0:
    rsp = await pcli.raw_request(
        "POST",
        "/papi/v1/bnb-transfer",
        params={"amount": p_bal_df.loc["BNB"]["umWalletBalance"], "transferSide": "FROM_UM"},
        auth=True,
    )
    print(rsp)

In [22]:
# repay cm/um negative balance
repay_resp = await pcli.raw_request("POST", "/papi/v1/repay-futures-negative-balance", params={}, auth=True)
print(repay_resp)
# 统一账户资金归集
collect_resp = await pcli.raw_request("POST", "/papi/v1/auto-collection", auth=True)
print(collect_resp)

{'msg': 'success'}
{'msg': 'success'}


In [23]:
# PM margin interest > 0 OR loan > 0
p_bal_resp = await pcli.get_balance()
p_bal_df = pd.DataFrame(p_bal_resp)
p_bal_df[(p_bal_df.crossMarginInterest.astype(float) > 0) | (p_bal_df.crossMarginBorrowed.astype(float) > 0)]

,asset,totalWalletBalance,crossMarginAsset,crossMarginBorrowed,crossMarginFree,crossMarginInterest,crossMarginLocked,umWalletBalance,umUnrealizedPNL,cmWalletBalance,cmUnrealizedPNL,updateTime,negativeBalance


In [24]:
# PM UM Balance != 0
p_bal_df[p_bal_df.umWalletBalance.astype(float) != 0]

,asset,totalWalletBalance,crossMarginAsset,crossMarginBorrowed,crossMarginFree,crossMarginInterest,crossMarginLocked,umWalletBalance,umUnrealizedPNL,cmWalletBalance,cmUnrealizedPNL,updateTime,negativeBalance


In [25]:
p_bal_df[p_bal_df.asset == "USDT"]

,asset,totalWalletBalance,crossMarginAsset,crossMarginBorrowed,crossMarginFree,crossMarginInterest,crossMarginLocked,umWalletBalance,umUnrealizedPNL,cmWalletBalance,cmUnrealizedPNL,updateTime,negativeBalance
1,USDT,158.26241723,158.26241723,0.0,158.26241723,0.0,0.0,0.0,-0.78108,0.0,0.0,1756039656598,0.0


In [71]:
# repay margin loan & interest
await pcli.raw_request(
    "POST",
    "/papi/v1/repayLoan",
    params={
        "asset": "USDT",
        "amount": 8,
    },
    auth=True,
)

{'tranId': 254688050863}

In [102]:
# pm margin loan, Use with CAUTION!!!

_use_with_caution = False
if _use_with_caution:
    rsp = await pcli.raw_request(
        "POST",
        "/papi/v1/marginLoan",
        params={
            "amount": "100000",
            "asset": "USDT",
        },
        auth=True,
    )
    print(rsp)

In [34]:
# transfer all assets from PM Margin -> Spot
p_bal_resp = await pcli.get_balance()

_use_with_caution = True
if _use_with_caution:
    for asset in p_bal_resp:
        if float(asset["crossMarginFree"]) > 0 :#and asset["asset"] != "USDT":
            print("Transfer {} {} from PM to SPOT".format(asset["crossMarginFree"], asset["asset"]))
            rsp = await scli.sapi_asset_transfer(type="PORTFOLIO_MARGIN_MAIN", asset=asset["asset"], amount=asset["crossMarginFree"])
            if "tranId" not in rsp:
                print("[ERROR]", rsp)
            await asyncio.sleep(0.5)

Transfer 0.00035089 BNB from PM to SPOT
Transfer 222.14869857 USDT from PM to SPOT


# Buy/Sell PM Margin asset

In [ ]:
# buy/sell pm margin asset, Use with CAUTION!!!

_use_with_caution = False

if _use_with_caution:
    rsp = await pcli.place_margin_order(
        symbol="USDCUSDT",
        side="BUY",
        type="MARKET",
        quantity="5",
    )
    print(rsp)

{'clientOrderId': '6Bq7hKH1VQiQVedi732428', 'cummulativeQuoteQty': '5.00200000', 'executedQty': '5.00000000', 'fills': [{'commission': '0.00000000', 'commissionAsset': 'BNB', 'price': '1.00040000', 'qty': '5.00000000'}], 'orderId': 859912908, 'origQty': '5.00000000', 'price': '0.00000000', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'side': 'BUY', 'status': 'FILLED', 'symbol': 'USDCUSDT', 'timeInForce': 'GTC', 'transactTime': 1742478735093, 'type': 'MARKET'}


# Buy/Sell spot asset

In [ ]:
# VWAP spot account, Use with CAUTION!!!

_use_with_caution = False
_n_orders = 1
_qty = 0.00224

if _use_with_caution:
    for i in range(_n_orders):
        rsp = await pcli.place_margin_order(
            symbol="BTCUSDT",
            side="SELL",
            type="MARKET",
            quantity=_qty,
        )
        print("[{}] order id = {}".format(i+1, rsp["orderId"]))
        await asyncio.sleep(1)

[1] order id = 44517664951


In [ ]:
# buy/sell spot asset, Use with CAUTION!!!

_use_with_caution = False
if _use_with_caution:
    rsp = await scli.spot_order(
        symbol="BNBUSDT",
        side="SELL",
        type="MARKET",
        quantity="0.162",
    )
    print(rsp)

{'symbol': 'BNBUSDT', 'orderId': 8229018379, 'orderListId': -1, 'clientOrderId': 'QlJ18Pi5SCBnl57gyMO28r', 'transactTime': 1749630650986, 'price': '0.00000000', 'origQty': '0.16200000', 'executedQty': '0.16200000', 'origQuoteOrderQty': '0.00000000', 'cummulativeQuoteQty': '108.80406000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'workingTime': 1749630650986, 'fills': [{'price': '671.63000000', 'qty': '0.16200000', 'commission': '0.00006317', 'commissionAsset': 'BNB', 'tradeId': 1075472526}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}


# Dust spot account

In [54]:
# get spot account balances
resp = await scli.get_spot_account()
bal_df = pd.DataFrame(resp["balances"])

resp = await scli.sapi_asset_dustable()
asset_l = list(map(lambda x: x["asset"], resp["details"]))
bal_df["will_dust"] = bal_df["asset"].apply(lambda x: x in asset_l)

In [55]:
bal_df[bal_df.will_dust == True]

,asset,free,locked,will_dust
179,DOGE,0.12100000,0.00000000,True
555,ORDI,0.19840000,0.00000000,True
557,TIA,0.00500000,0.00000000,True
582,WIF,0.00374000,0.00000000,True
608,HMSTR,245.54,0.00,True
610,NEIRO,0.57,0.00,True
617,PNUT,0.56950000,0.00000000,True
618,ACT,0.50500000,0.00000000,True


In [56]:
resp = await scli.sapi_asset_dust(asset=','.join(asset_l))
resp

{'totalTransfered': '0.00534385',
 'totalServiceCharge': '0.00010684',
 'transferResult': [{'tranId': 240696741070,
   'fromAsset': 'ACT',
   'amount': '0.505',
   'transferedAmount': '0.00016816',
   'serviceChargeAmount': '0.00000336',
   'operateTime': 1738833231395},
  {'tranId': 240696741070,
   'fromAsset': 'NEIRO',
   'amount': '0.57',
   'transferedAmount': '0.00000034',
   'serviceChargeAmount': '0',
   'operateTime': 1738833231385},
  {'tranId': 240696741070,
   'fromAsset': 'WIF',
   'amount': '0.00374',
   'transferedAmount': '0.00000525',
   'serviceChargeAmount': '0.0000001',
   'operateTime': 1738833231386},
  {'tranId': 240696741070,
   'fromAsset': 'HMSTR',
   'amount': '245.54',
   'transferedAmount': '0.00073093',
   'serviceChargeAmount': '0.00001461',
   'operateTime': 1738833231395},
  {'tranId': 240696741070,
   'fromAsset': 'PNUT',
   'amount': '0.5695',
   'transferedAmount': '0.00017757',
   'serviceChargeAmount': '0.00000355',
   'operateTime': 1738833231395}

# UM Position Operations

In [61]:
# Place UM Order
_use_with_caution = True
if _use_with_caution:
    rsp = await pcli.place_um_order(
        symbol="BTCUSDT",
        side="BUY",
        type="MARKET",
        quantity=0.007,
    )
    print("order_id={} resp={}".format(rsp["orderId"], rsp))

order_id=704325701344 resp={'symbol': 'BTCUSDT', 'side': 'BUY', 'executedQty': '0.000', 'orderId': 704325701344, 'goodTillDate': 0, 'avgPrice': '0.00000', 'origQty': '0.007', 'clientOrderId': 'qGSNIOF1CpEmUolbF1P16', 'positionSide': 'BOTH', 'cumQty': '0.000', 'updateTime': 1749622439302, 'type': 'MARKET', 'reduceOnly': False, 'price': '0.00', 'cumQuote': '0.00000', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'timeInForce': 'GTC', 'status': 'NEW', 'priceMatch': 'NONE'}


In [95]:
# Get open orders
rsp = await pcli.get_um_open_orders("LUNA2USDT")
pprint(rsp)

[]


In [189]:
# Cancel open orders
rsp = await pcli.cancel_um_order("COWUSDT", clientOid="tuNwX4olnc8C1cHn69shOyJ30mDPphMe")
pprint(rsp)

{'avgPrice': '0',
 'clientOrderId': 'tuNwX4olnc8C1cHn69shOyJ30mDPphMe',
 'cumQty': '0',
 'cumQuote': '0.0000000',
 'executedQty': '0',
 'goodTillDate': 0,
 'orderId': 187626710,
 'origQty': '63',
 'positionSide': 'BOTH',
 'price': '0.3772000',
 'priceMatch': 'NONE',
 'reduceOnly': False,
 'selfTradePreventionMode': 'NONE',
 'side': 'BUY',
 'status': 'CANCELED',
 'symbol': 'COWUSDT',
 'timeInForce': 'GTX',
 'type': 'LIMIT',
 'updateTime': 1737046590953}


In [117]:
# VWAP PM UM positions, Use with CAUTION!!!

_use_with_caution = False
_n_orders = 50
_qty = 20

if _use_with_caution:
    for i in range(_n_orders):
        rsp = await pcli.place_um_order(
            symbol="OMUSDT",
            side="BUY",
            type="MARKET",
            quantity=_qty,
        )
        print("[{}] order id = {}".format(i+1, rsp["orderId"]))
        await asyncio.sleep(1)

[1] order id = 4489242053
[2] order id = 4489242662
[3] order id = 4489243221
[4] order id = 4489243768
[5] order id = 4489244619
[6] order id = 4489244964
[7] order id = 4489245561
[8] order id = 4489246119
[9] order id = 4489246737
[10] order id = 4489247006
[11] order id = 4489247657
[12] order id = 4489248083
[13] order id = 4489248546
[14] order id = 4489248742
[15] order id = 4489249309
[16] order id = 4489249882
[17] order id = 4489250173
[18] order id = 4489250644
[19] order id = 4489251296
[20] order id = 4489252115
[21] order id = 4489252664
[22] order id = 4489253147
[23] order id = 4489254304
[24] order id = 4489255046
[25] order id = 4489255843
[26] order id = 4489256612
[27] order id = 4489257041
[28] order id = 4489257582
[29] order id = 4489258520
[30] order id = 4489259640
[31] order id = 4489260232
[32] order id = 4489260695
[33] order id = 4489261130
[34] order id = 4489261790
[35] order id = 4489262328
[36] order id = 4489263115
[37] order id = 4489263775
[38] order

# VWAP 对冲平仓

In [516]:
_use_with_caution = True
meta_symbol = "CHEEMS_USDT"
total_meta_qty = 18112601000
order_meta_qty = 72450000
order_cnt = total_meta_qty // order_meta_qty
actual_qty = order_meta_qty * order_cnt

sp_inst = inst_mngr.get_inst_by_meta_symbol(Exchange.BINANCE, InstType.SP, meta_symbol)
lps_inst = inst_mngr.get_inst_by_meta_symbol(Exchange.BINANCE, InstType.LPS, meta_symbol)

try:
    if order_meta_qty % sp_inst.price_multiplier == 0:
        sp_order_qty = int(order_meta_qty / sp_inst.price_multiplier)
        assert sp_order_qty > sp_inst.min_order_size, "SP min order size mismatch!"
        sp_order_arg = {
            "symbol": sp_inst.exchange_symbol,
            "side": "SELL",
            "type": "MARKET",
            "quantity": str(sp_order_qty),
        }
    else:
        raise ValueError("Invalid Meta Qty for SP")
    if order_meta_qty % lps_inst.price_multiplier == 0:
        lps_order_qty = int(order_meta_qty / lps_inst.price_multiplier)
        assert lps_order_qty > lps_inst.min_order_size, "LPS min order size mismatch!"
        lps_order_arg = {
            "symbol": lps_inst.exchange_symbol,
            "side": "BUY",
            "type": "MARKET",
            "reduceOnly": True,
            "quantity": str(lps_order_qty),
        }
    else:
        raise ValueError("Invalid Meta Qty for LPS")
except Exception as e:
    _vwap_config_validated = False
    print("Validation Failed!")
    print(e)
else:
    _vwap_config_validated = True
    print("Validation Success!")
    print(
        "total_meta_qty={} order_meta_qty={} order_cnt={} actual_qty={}".format(
            total_meta_qty, order_meta_qty, order_cnt, order_meta_qty * order_cnt
        )
    )
    print("sp_order_arg={}".format(sp_order_arg))
    print("lps_order_arg={}".format(lps_order_arg))

Validation Success!
total_meta_qty=18112601000 order_meta_qty=72450000 order_cnt=250 actual_qty=18112500000
sp_order_arg={'symbol': '1000CHEEMSUSDT', 'side': 'SELL', 'type': 'MARKET', 'quantity': '72450'}
lps_order_arg={'symbol': '1000CHEEMSUSDT', 'side': 'BUY', 'type': 'MARKET', 'reduceOnly': True, 'quantity': '72450'}


In [517]:
if _use_with_caution and _vwap_config_validated:
    for cnt in range(order_cnt):
        print(f"[{cnt+1}/{order_cnt}] SELL SP: {sp_order_qty} {sp_inst.exchange_symbol}")
        sp_rsp = await pcli.place_margin_order(**sp_order_arg)
        print(sp_rsp)

        print(f"[{cnt+1}/{order_cnt}] BUY LPS: {lps_order_qty} {lps_inst.exchange_symbol}")
        lps_rsp = await pcli.place_um_order(**lps_order_arg)
        print(lps_rsp)

        _sleep = random.random() * 1 + 1
        print(f"sleep {_sleep:.2f}s")
        await asyncio.sleep(_sleep)

[1/250] SELL SP: 72450 1000CHEEMSUSDT
{'clientOrderId': 'Cr5ldGxGc1INoe9ZX1d298', 'cummulativeQuoteQty': '102.80655000', 'executedQty': '72450.00', 'fills': [{'commission': '0.00001746', 'commissionAsset': 'BNB', 'price': '0.00141900', 'qty': '21141.00'}, {'commission': '0.00004239', 'commissionAsset': 'BNB', 'price': '0.00141900', 'qty': '51309.00'}], 'orderId': 121816285, 'origQty': '72450.00', 'price': '0.00000000', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'side': 'SELL', 'status': 'FILLED', 'symbol': '1000CHEEMSUSDT', 'timeInForce': 'GTC', 'transactTime': 1749625646655, 'type': 'MARKET'}
[1/250] BUY LPS: 72450 1000CHEEMSUSDT
{'symbol': '1000CHEEMSUSDT', 'side': 'BUY', 'executedQty': '0', 'orderId': 840168412, 'goodTillDate': 0, 'avgPrice': '0.0000000', 'origQty': '72450', 'clientOrderId': 'G1lGFdvJACfVp0JTZI0783', 'positionSide': 'BOTH', 'cumQty': '0', 'updateTime': 1749625646671, 'type': 'MARKET', 'reduceOnly': True, 'price': '0.0000000', 'cumQuote': '0.0000000', 'selfTradePreve

# 跨子账户转账

In [28]:
jw_acct_email_map = {
    "4abnjwarb01": "chrix001_virtual@7zozg3e3noemail.com",
    "4abnjwarb02": "chrix002_virtual@exziukkmnoemail.com",
    "4abnjwarb03": "chrix003_virtual@ppm6h3zmnoemail.com",
}

from_acct = "4abnjwarb02"
to_acct = "4abnjwarb01"

trsf_scli: BinanceSpotRestClient = get_rest_client(Exchange.BINANCE, InstType.SP, account=from_acct)
trsf_pcli: BinanceUnifiedRestClient = get_rest_client(Exchange.BINANCEUNIFIED, InstType.SP, account=from_acct)

In [35]:
asset = "USDT"

trsf_p_bal = await trsf_pcli.get_balance()
trsf_p_bal_df = pd.DataFrame(trsf_p_bal).set_index("asset")
trsf_p_bal_df.loc[asset]

totalWalletBalance               0.0
crossMarginAsset                 0.0
crossMarginBorrowed              0.0
crossMarginFree                  0.0
crossMarginInterest              0.0
crossMarginLocked                0.0
umWalletBalance                  0.0
umUnrealizedPNL                  0.0
cmWalletBalance                  0.0
cmUnrealizedPNL                  0.0
updateTime             1749625034948
negativeBalance                  0.0
Name: USDT, dtype: object

In [36]:
resp = await trsf_scli.get_spot_account()
s_bal_df = pd.DataFrame(resp["balances"]).set_index("asset")
s_bal_df.free = s_bal_df.free.astype(float)
s_bal_df.loc[asset]

free     0.0000000000
locked     0.00000000
Name: USDT, dtype: object

In [34]:
_use_with_caution = True
if _use_with_caution:
    trsf_resp = await trsf_scli.sapi_sub_account_transfer_sub2sub(
        toEmail=jw_acct_email_map[to_acct],
        asset=asset,
        amount=Decimal(s_bal_df.loc[asset].free),
    )
    pprint(trsf_resp)

{'txnId': 268207749849}
